# 大模型提示词工程

## 1. 提示词基础

提示词工程也称为“In-Context Prompting”，其实就是我们使用大模型的最最基本的方式：对话。我们向大模型提出要求，大模型理解我们的意图，给出答案。
在这个过程中，我们通过改变话术就可以影响大模型输出答案的准确性，这个不断与大模型对话不断校准获得我们想要的答案的过程，就是提示词工程。

下面我们来看一个简单的案例，首先我们定义访问函数如下：

In [29]:
import os
from dotenv import load_dotenv

## for MacOS users
filePath = os.path.abspath(os.path.expanduser(os.path.expandvars("~/.zshrc")))
load_dotenv(filePath)

Python-dotenv could not parse statement starting at line 7
Python-dotenv could not parse statement starting at line 8
Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 16


True

## 制作一个LLM查询客户端

In [30]:
import sys
from langchain_community.llms import Tongyi

def Q(question):
    llm = Tongyi(streaming=True)
    llm.model_name = 'qwen-turbo'
    for chunk in llm.stream(question):
        print(chunk, end="", flush=True)

In [31]:
# Q("请续写儿歌，第一句是：太阳当空照")

In [32]:
Q("海湾战争的起因是什么？")

海湾战争是以1990年8月2日伊拉克入侵科威特开始的，是二战后参战国家最多的一场局部战争。其直接起因是伊拉克和科威特之间关于边界纠纷和石油问题的争执。伊拉克认为科威特的石油资源丰富，对其领土有野心；而科威特则寻求国际社会的支持来维护其独立和主权。在联合国安理会未能达成和平解决协议的情况下，伊拉克悍然出兵占领了科威特，引发了国际社会的强烈谴责和反对。以美国为首的多国联盟随后组成联军，对伊拉克进行军事干预，最终迫使伊拉克撤军，恢复了科威特的主权。这场战争也展示了现代高科技战争的特点，包括大规模使用空中打击和电子战。

In [33]:
Q("请把上面的话翻译成英文")

Please translate the above statement into English.

### 1.1 实现简单多轮对话

上述介绍的零提示词法并不涉及记忆能力，也就是说两个问句之间没有内在联系。

但是，当我们在使用大模型应用时通常会感觉对话很自然，这是因为真实大模型应用是多轮对话模式，大模型是有记忆的。

如何实现这一点呢？我们考虑大模型对话的基础仍然是简单提示词对话。因此，如果我们把上一轮的“对话内容”巧妙加入到当前对话中，我们就应该能实现和大模型的多轮对话。

当然，这个对话的历史长度取决于大模型可输入的tokens总量。

接下来，我们尝试考虑上下文，实现一个多轮对话。请尝试如下操作：

In [2]:
import json
from langchain_community.llms import Tongyi

class MyAgent:
    def __init__(self, mem_len=10, model_name = 'qwen-max'):
        self.memory = []
        self.men_len = mem_len
        self.model_name = model_name

    def QA(self, question, new_context=False):
        llm = Tongyi(streaming=True)
        llm.model_name = self.model_name

        if(new_context==True):
            self.memory = []
        prompt = {"question":question, "history":self.memory}
        
        ans=""
        for chunk in llm.stream(json.dumps(prompt , ensure_ascii=False)):
            ans += chunk
            print(chunk, end="", flush=True)
        self.memory.append((question, ans))

        if(len(self.memory)>self.men_len):
            self.memory.pop(0)

        return ans

llm = MyAgent()

In [3]:
llm.QA("阿里云ECS能做什么？")

阿里云ECS（Elastic Compute Service）是阿里云提供的弹性计算服务，它允许用户在云端获取和管理计算资源。通过使用ECS，您可以快速部署和运行各种应用程序、网站或服务，而无需投资和维护底层硬件。以下是阿里云ECS可以做的事情：

1. **托管网站和应用**：您可以部署和运行个人、企业或电商网站，以及各种Web应用程序，如内容管理系统（CMS）、论坛、博客等。

2. **大数据处理**：ECS可提供弹性的计算能力，支持大规模数据处理和分析任务，如Hadoop、Spark等分布式计算框架。

3. **机器学习与人工智能**：ECS可以作为机器学习模型训练和推理的平台，支持TensorFlow、PyTorch等AI框架。

4. **游戏服务器**：游戏开发者可以使用ECS来托管游戏服务器，提供稳定的游戏体验。

5. **软件开发和测试**：ECS可以快速创建和销毁开发环境，用于软件开发、调试和测试。

6. **媒体转码**：阿里云ECS可以配合其他服务，如媒体转换服务，进行视频编码和转码工作。

7. **云计算服务**：作为基础设施即服务（IaaS）的一部分，ECS可以作为其他阿里云服务的基础，如数据库服务、CDN、负载均衡等。

8. **在线教育**：支持实时音视频互动，构建在线课堂和教育平台。

9. **容器服务**：ECS可以与Kubernetes、Docker等容器技术结合，实现更高效的应用部署和管理。

10. **高可用和容灾**：通过阿里云的负载均衡和多区域部署，实现业务的高可用性和灾难恢复。

总的来说，阿里云ECS是一个功能强大的计算平台，可以满足各种企业和个人的计算需求，从简单的网站托管到复杂的云原生应用部署，都能提供灵活、可扩展的解决方案。

In [4]:
llm.QA("ECS 能部署应用吗？")

是的，ECS 可以部署应用。您可以将各种类型的应用程序部署到阿里云ECS实例上，包括但不限于Web应用、大数据应用、机器学习模型、游戏服务器、软件开发环境等。您可以直接在ECS实例上安装所需的应用服务器、数据库和其他软件，然后配置和管理您的应用。此外，通过容器服务（如Kubernetes或Docker），ECS还可以支持更加现代化和自动化应用的部署和管理。

In [5]:
llm.QA("把ECS特性的第二项翻译成英文")

The second feature of ECS is "Hosting websites and applications".

In [6]:
llm.memory

[('阿里云ECS能做什么？',
  '阿里云ECS（Elastic Compute Service）是阿里云提供的弹性计算服务，它允许用户在云端获取和管理计算资源。通过使用ECS，您可以快速部署和运行各种应用程序、网站或服务，而无需投资和维护底层硬件。以下是阿里云ECS可以做的事情：\n\n1. **托管网站和应用**：您可以部署和运行个人、企业或电商网站，以及各种Web应用程序，如内容管理系统（CMS）、论坛、博客等。\n\n2. **大数据处理**：ECS可提供弹性的计算能力，支持大规模数据处理和分析任务，如Hadoop、Spark等分布式计算框架。\n\n3. **机器学习与人工智能**：ECS可以作为机器学习模型训练和推理的平台，支持TensorFlow、PyTorch等AI框架。\n\n4. **游戏服务器**：游戏开发者可以使用ECS来托管游戏服务器，提供稳定的游戏体验。\n\n5. **软件开发和测试**：ECS可以快速创建和销毁开发环境，用于软件开发、调试和测试。\n\n6. **媒体转码**：阿里云ECS可以配合其他服务，如媒体转换服务，进行视频编码和转码工作。\n\n7. **云计算服务**：作为基础设施即服务（IaaS）的一部分，ECS可以作为其他阿里云服务的基础，如数据库服务、CDN、负载均衡等。\n\n8. **在线教育**：支持实时音视频互动，构建在线课堂和教育平台。\n\n9. **容器服务**：ECS可以与Kubernetes、Docker等容器技术结合，实现更高效的应用部署和管理。\n\n10. **高可用和容灾**：通过阿里云的负载均衡和多区域部署，实现业务的高可用性和灾难恢复。\n\n总的来说，阿里云ECS是一个功能强大的计算平台，可以满足各种企业和个人的计算需求，从简单的网站托管到复杂的云原生应用部署，都能提供灵活、可扩展的解决方案。'),
 ('ECS 能部署应用吗？',
  '是的，ECS 可以部署应用。您可以将各种类型的应用程序部署到阿里云ECS实例上，包括但不限于Web应用、大数据应用、机器学习模型、游戏服务器、软件开发环境等。您可以直接在ECS实例上安装所需的应用服务器、数据库和其他软件，然后配置和管理您的应用。此外，通过容器服务（如Kubernetes或Docker），ECS还可以支持更加现代化和自动化应用的部署